The following MIT license only applies to the code, and not to the text and images.

# MIT License

Copyright (c) 2024 Taha Selim, Alain Chancé, MolKet

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

|||
|-|-|
|**Author:** |Taha Selim and Alain Chancé|
|**Date:** |July 10th, 2024|
|**Tutorial:** |**Version 0** <br/>  ML wavefunction generator for a diatomic system givin a vibrational frequency and a intermolecular 1D potential. |
|**MolKet's package version:** |0.2|
|**Contacts:** | Taha Selim, tahaselim@molket.io 
|             | Alain Chancé, alain.chance@gmail.com|


In [1]:

# get julia version 
versioninfo()

Julia Version 1.11.2
Commit 5e9a32e7af2 (2024-12-01 20:02 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin24.0.0)
  CPU: 8 × Apple M1 Pro
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, apple-m1)
Threads: 1 default, 0 interactive, 1 GC (on 6 virtual cores)


In [2]:
# Load MolKet's modules and libraries

include("../molket.jl")


Load quantum gates constructor
Load Tensor module: QTensor.jl
Load quantum gates constructor
Load quantum_circuit constructor
Load quantum_circuit constructor
Load quantum gates constructor
Load Tensor module: QTensor.jl
Load quantum gates constructor
Load quantum_circuit constructor
Load quantum gates constructor
Load Tensor module: QTensor.jl
Load quantum gates constructor
Load quantum_circuit constructor
Load quantum gates constructor
Load Tensor module: QTensor.jl
Load quantum gates constructor
Load quantum_circuit constructor
Load quantum_circuit constructor
Load quantum_circuit constructor
Load quantum_circuit constructor
Load quantum_circuit constructor
Load ascii 2Darray


In [3]:
# using Libdl

In [4]:
# test the shared object 
#Libdl.dlopen("HeCO2potv2.so")

Libdl.dlopen("../lib_potential/HeCO2potv2.so")


Ptr{Nothing} @0x00000000957050d0

In [5]:
# Inputs
R0 = 5.7       # CO2-He distance
QQ = 0.0       # Dimensionless normal coordinate
theta = 90.0    # Angle with z-axis
phi = 0      # Angle in xy-plane
vpot = Ref{Cdouble}(0.0)  # Output: Potential energy

# Call the subroutine
ccall((:heco2potv2_, "../lib_potential/HeCO2potv2.so"), Cvoid, 
      (Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}),
      R0, QQ, theta, phi, vpot)

# Print the result
println("Calculated potential: ", vpot[]/cm1)


Calculated potential: 44.11877388093571


In the following cell, we will test the potential on a given grid of points in $Q$, $R$, $\theta$ and $\phi$ and plot the potential.

In [6]:
# Retrieve reference data from file

filename = "../lib_potential/Vpot_heco2_refdata.dat"
 pot_data = read_mixed_data(filename)
 println(pot_data)

Any[Any[1.0, 6.39048431790434, 0.852520652115345, 1.63644068089431, 5.85072821410867, -38.77195013853806, "-0.3877195013853705E+02********************"], Any[2.0, 8.79615684668534, -0.465971812605858, 0.367309045633237, 5.59910107307967, -17.52061247339478, "-0.1752061247339592E+02********************"], Any[3.0, 9.54409375670366, 1.96406215429306, 1.16891764042022, 4.43707493105516, -4.648651061978663, "-0.4648651061978669E+01********************"], Any[4.0, 10.2592325580772, -1.1046179458499, 1.08604020058505, 1.47065720263843, -2.278837065922246, "-0.2278837065922187E+01********************"], Any[5.0, 11.8822548517492, 0.409278564155102, 1.79545059535835, 0.345342112062281, -0.1453827988383787, "-0.1453827988382038E+00********************"], Any[6.0, 13.4812615595292, -1.93289755284786, 2.93594742258905, 3.52478135752025, -0.004326054469524383, "-0.4326054469093559E-02********************"], Any[7.0, 16.1922277247068, -0.481204651296139, 0.962067770990025, 2.43790844202072, 9.88781

In [7]:
# write a code to loop over R as second column, Q as third column, theta as fourth column and phi as fifth column and calculate the potential energy for each set of values

# create a matrix of R, Q, theta, phi, vpot_ref and vpot
Vpot_mat = def_matrix(length(pot_data), 6)

for i in 1:length(pot_data)
    R0 = pot_data[i][2]
    QQ = pot_data[i][3]
    theta = pot_data[i][4]
    phi = pot_data[i][5]
    vpot_ref = pot_data[i][6]
    vpot = Ref{Cdouble}(0.0)  # Output: Potential energy

    # Call the subroutine
    ccall((:heco2potv2_, "HeCO2potv2.so"), Cvoid,
          (Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}),
          R0, QQ, theta, phi, vpot)
    
    rel_error = abs(vpot[]/cm1 - vpot_ref)/abs(vpot_ref)

    # Print the result
    #println("Calculated potential: ", vpot[]/cm1)

    # create a matrix of R, Q, theta, phi, vpot_ref and vpot
    println("R0 = ", R0, " QQ = ", QQ, " theta = ", theta, " phi = ", 
    phi, " vpot_ref = ", vpot_ref, " vpot = ", vpot[]/cm1, " relative error = ", rel_error)

end

R0 = 6.39048431790434 QQ = 0.852520652115345 theta = 1.63644068089431 phi = 5.85072821410867 vpot_ref = -38.77195013853806 vpot = -38.77195013853807 relative error = 1.8326205754965206e-16
R0 = 8.79615684668534 QQ = -0.465971812605858 theta = 0.367309045633237 phi = 5.59910107307967 vpot_ref = -17.52061247339478 vpot = -17.520612473394785 relative error = 2.0277337246031386e-16
R0 = 9.54409375670366 QQ = 1.96406215429306 theta = 1.16891764042022 phi = 4.43707493105516 vpot_ref = -4.648651061978663 vpot = -4.648651061978663 relative error = 0.0
R0 = 10.2592325580772 QQ = -1.1046179458499 theta = 1.08604020058505 phi = 1.47065720263843 vpot_ref = -2.278837065922246 vpot = -2.2788370659222457 relative error = 1.9487536712956684e-16
R0 = 11.8822548517492 QQ = 0.409278564155102 theta = 1.79545059535835 phi = 0.345342112062281 vpot_ref = -0.1453827988383787 vpot = -0.14538279883837868 relative error = 1.909137520903325e-16
R0 = 13.4812615595292 QQ = -1.93289755284786 theta = 2.93594742258905

Let's make some cuts in the potential to see the effect of the different parameters. First, let's evaluate the potential and make a surface plot of the potential in $\text{cm}^{-1}$ as a function of $R$ and $\theta$ while keeping $Q=0$ and $\phi=0$ constant.

In [8]:
# make a surface plot of R and theta with vpot as the z-axis
# First, R gird 
R = collect(4.0:0.1:6.0)
# Second, theta grid
theta = collect(0.0:10.0:180.0)
# Third, create a matrix of R and theta
Vpot_Rtheta = def_matrix(length(R),length(theta))

# create a loop to call the potential subroutine 

for i in 1:length(R)
    for j in 1:length(theta)
        R0 = R[i]
        QQ = 0.0
        theta0 = theta[j]
        phi = 0
        vpot = Ref{Cdouble}(0.0)  # Output: Potential energy

        # Call the subroutine
        ccall((:heco2potv2_, "HeCO2potv2.so"), Cvoid,
              (Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}, Ref{Cdouble}),
              R0, QQ, theta0, phi, vpot)
        
        Vpot_Rtheta[i,j] = vpot[]/cm1
    end
end

In [9]:
# print the matrix
println(Vpot_Rtheta)

Union{Float64, Int64, ComplexF64}[44576.459114629746 30936.580208138254 3150.4172984085903 1528.1087549717122 10849.95923441141 43509.18441531382 43404.71434576135 9203.526411565326 1343.2094674568227 3838.674604635324 34786.38060647703 44483.265809197386 26783.970299476205 2636.1957612505794 1707.328226737813 12944.645326105514 43561.438525155 43027.95308317503 7860.354401825917; 43551.80504763278 26449.960485768923 2603.244276318581 1187.5705993034564 9084.46308754159 43564.66239796206 42843.35235715779 7678.883371695069 1050.9400947720746 3159.442259797886 30309.169063210076 43589.75215640774 22596.119467056706 2177.77272737582 1332.3262994519494 10841.067138251337 43874.45404668239 41575.81438169739 6518.471398553053; 42577.970237786496 22050.744587898702 2169.53433433822 889.0640693557775 7655.057402800537 42614.20254232789 41188.301660829915 6405.339580964621 806.115681588497 2595.4006915608716 25652.860604717865 42695.857009700565 18707.390207673358 1823.7040330815075 1000.45416

Now, let's create a surface plot. 

In [12]:
# create a surface plot of R and theta with Vpot_Rtheta 

plot(R, theta, Vpot_Rtheta, st = :surface, xlabel = "R", ylabel = "theta", zlabel = "Vpot",
    title = "HeCO2 potential energy surface plot", camera = (30, 30))


In [11]:

# ... your code to generate R, theta, and Vpot_Rtheta ...

# Sample the data with a step
step = 5  # Adjust this value: larger step means fewer points

sampled_R = R[1:step:end, 1:step:end]  # Apply to both dimensions if R is 2D
sampled_theta = theta[1:step:end, 1:step:end] # Similar to R
sampled_Vpot_Rtheta = Vpot_Rtheta[1:step:end, 1:step:end]

# Create the surface plot with the sampled data
plot(sampled_R, sampled_theta, sampled_Vpot_Rtheta, 
     st = :surface, 
     xlabel = "R", 
     ylabel = "theta", 
     zlabel = "Vpot",
     title = "HeCO2 potential energy surface plot", 
     camera = (30, 30))